In [30]:
import pandas as pd
import numpy as np

#countvectorizer model
import nltk
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

#tf-idf model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel



In [31]:
df = pd.read_csv("../data/goodreads_final_bagowords.csv")
df.head()

,url,bookTitle,bookImage,bookDesc,bookRating,ratingCount,reviewCount,Genre,pageCount,Author,lang,title_key_words,clean_keywords,sentiment_polarity,sentiment_subjectivity,genre_bag,bag_of_words
0,https://www.goodreads.com/book/show/2767052-th...,The Hunger Games,https://i.gr-assets.com/images/S/compressed.ph...,"Could you survive on your own in the wild, wit...",4.32,6717635,176054,Young Adult,374,Suzanne Collins,en,"['hunger', 'games']","['could', 'survive', 'wild', 'every', 'one', '...",0.101623,0.484921,['youngadult'],youngadult hunger games could survive wild eve...
1,https://www.goodreads.com/book/show/2.Harry_Po...,Harry Potter and the Order of the Phoenix,https://i.gr-assets.com/images/S/compressed.ph...,There is a door at the end of a silent corrido...,4.50,2668409,45724,Fantasy,870,J.K. Rowling,en,"['harry', 'potter', 'order', 'phoenix']","['door', 'end', 'silent', 'corridor', 'hauntin...",-0.087273,0.420909,['fantasy'],fantasy harry potter order phoenix door end si...
2,https://www.goodreads.com/book/show/2657.To_Ki...,To Kill a Mockingbird,https://i.gr-assets.com/images/S/compressed.ph...,The unforgettable novel of a childhood in a sl...,4.28,4772918,95595,Classics,324,Harper Lee,en,"['kill', 'mockingbird']","['unforgettable', 'novel', 'childhood', 'sleep...",0.165686,0.368067,['classics'],classics kill mockingbird unforgettable novel ...
3,https://www.goodreads.com/book/show/1885.Pride...,Pride and Prejudice,https://i.gr-assets.com/images/S/compressed.ph...,Alternate cover edition of ISBN 9780679783268S...,4.27,3206070,74020,Classics,279,Jane Austen,en,"['pride', 'prejudice']","['alternate', 'cover', 'edition', 'isbn', 'sin...",0.475556,0.680000,['classics'],classics pride prejudice alternate cover editi...
4,https://www.goodreads.com/book/show/41865.Twil...,Twilight,https://i.gr-assets.com/images/S/compressed.ph...,About three things I was absolutely positive.F...,3.61,5231000,107619,Young Adult,501,Stephenie Meyer,en,['twilight'],"['three', 'things', 'absolutely', 'positive', ...",0.240000,0.620000,['youngadult'],youngadult twilight three things absolutely po...


## EDA

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8863 entries, 0 to 8862
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   url                     8863 non-null   object 
 1   bookTitle               8863 non-null   object 
 2   bookImage               8863 non-null   object 
 3   bookDesc                8863 non-null   object 
 4   bookRating              8863 non-null   float64
 5   ratingCount             8863 non-null   int64  
 6   reviewCount             8863 non-null   int64  
 7   Genre                   8863 non-null   object 
 8   pageCount               8863 non-null   int64  
 9   Author                  8863 non-null   object 
 10  lang                    8863 non-null   object 
 11  title_key_words         8863 non-null   object 
 12  clean_keywords          8863 non-null   object 
 13  sentiment_polarity      8863 non-null   float64
 14  sentiment_subjectivity  8863 non-null   

In [33]:
df["bookTitle"].nunique()

8863

In [34]:
df["Author"].nunique()

3896

In [35]:
df["Genre"].nunique()

46

## Count Vectorizer

In [36]:
#CountVectorizer model
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)

indices = pd.Series(df['bookTitle'])

[[1.         0.01299688 0.02515773 ... 0.02151657 0.03822354 0.07470179]
 [0.01299688 1.         0.         ... 0.04474374 0.0264953  0.03106849]
 [0.02515773 0.         1.         ... 0.02165233 0.0384647  0.04510386]
 ...
 [0.02151657 0.04474374 0.02165233 ... 1.         0.         0.02571722]
 [0.03822354 0.0264953  0.0384647  ... 0.         1.         0.09137174]
 [0.07470179 0.03106849 0.04510386 ... 0.02571722 0.09137174 1.        ]]


In [37]:
test = count_matrix.toarray()
test


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [38]:
len(test[0])

46788

In [39]:
sum(test[0])

76

In [40]:
def recommend(bookTitle, cosine_sim = cosine_sim):
    recommended_book = []
    idx = indices[indices == bookTitle].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_book.append(list(df['bookTitle'])[i])
        
    return recommended_book

In [41]:
recommend('To Kill a Mockingbird')

['Go Set a Watchman',
 'Lonesome Dove',
 'The Chronicles of Prydain',
 'Our Town',
 'Lady of Hay',
 'The Grapes of Wrath',
 'Manchild in the Promised Land',
 'The Secret Garden',
 'Story of a Soul: The Autobiography of St. Thérèse of Lisieux',
 'Jimmy Corrigan: The Smartest Kid on Earth']

## TF- IDF

In [42]:
df.head()

,url,bookTitle,bookImage,bookDesc,bookRating,ratingCount,reviewCount,Genre,pageCount,Author,lang,title_key_words,clean_keywords,sentiment_polarity,sentiment_subjectivity,genre_bag,bag_of_words
0,https://www.goodreads.com/book/show/2767052-th...,The Hunger Games,https://i.gr-assets.com/images/S/compressed.ph...,"Could you survive on your own in the wild, wit...",4.32,6717635,176054,Young Adult,374,Suzanne Collins,en,"['hunger', 'games']","['could', 'survive', 'wild', 'every', 'one', '...",0.101623,0.484921,['youngadult'],youngadult hunger games could survive wild eve...
1,https://www.goodreads.com/book/show/2.Harry_Po...,Harry Potter and the Order of the Phoenix,https://i.gr-assets.com/images/S/compressed.ph...,There is a door at the end of a silent corrido...,4.50,2668409,45724,Fantasy,870,J.K. Rowling,en,"['harry', 'potter', 'order', 'phoenix']","['door', 'end', 'silent', 'corridor', 'hauntin...",-0.087273,0.420909,['fantasy'],fantasy harry potter order phoenix door end si...
2,https://www.goodreads.com/book/show/2657.To_Ki...,To Kill a Mockingbird,https://i.gr-assets.com/images/S/compressed.ph...,The unforgettable novel of a childhood in a sl...,4.28,4772918,95595,Classics,324,Harper Lee,en,"['kill', 'mockingbird']","['unforgettable', 'novel', 'childhood', 'sleep...",0.165686,0.368067,['classics'],classics kill mockingbird unforgettable novel ...
3,https://www.goodreads.com/book/show/1885.Pride...,Pride and Prejudice,https://i.gr-assets.com/images/S/compressed.ph...,Alternate cover edition of ISBN 9780679783268S...,4.27,3206070,74020,Classics,279,Jane Austen,en,"['pride', 'prejudice']","['alternate', 'cover', 'edition', 'isbn', 'sin...",0.475556,0.680000,['classics'],classics pride prejudice alternate cover editi...
4,https://www.goodreads.com/book/show/41865.Twil...,Twilight,https://i.gr-assets.com/images/S/compressed.ph...,About three things I was absolutely positive.F...,3.61,5231000,107619,Young Adult,501,Stephenie Meyer,en,['twilight'],"['three', 'things', 'absolutely', 'positive', ...",0.240000,0.620000,['youngadult'],youngadult twilight three things absolutely po...


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [44]:
#https://practicaldatascience.co.uk/data-science/how-to-create-content-recommendations-using-tf-idf
#bag of words is already made, but this will also down-weight common words that appear across documents
tfidf = TfidfVectorizer(stop_words='english')

# numbers to calculate similarities
tfidf_matrix = tfidf.fit_transform(df['bag_of_words'])

#calculate cosine matrix
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_similarities)

indices = pd.Series(df['bookTitle'])

[[1.         0.00402374 0.0138527  ... 0.00703557 0.00161726 0.01776498]
 [0.00402374 1.         0.         ... 0.03376514 0.01499929 0.01502661]
 [0.0138527  0.         1.         ... 0.         0.01281669 0.01778946]
 ...
 [0.00703557 0.03376514 0.         ... 1.         0.         0.01030452]
 [0.00161726 0.01499929 0.01281669 ... 0.         1.         0.03148951]
 [0.01776498 0.01502661 0.01778946 ... 0.01030452 0.03148951 1.        ]]


In [50]:
#get recommendations based on tf-idf and cosine_similarities

def recommend_tf(bookTitle, cosine_similarities = cosine_similarities):
    
    recommended_book = []
    
    idx = indices[indices == bookTitle].index[0]
    
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)
    
    top_10_indices = list(score_series.iloc[1:21].index)
    
    for i in top_10_indices:
        recommended_book.append(list(df['bookTitle'])[i])
        
    return recommended_book

In [51]:
recommend_tf('The Lightning Thief')

['Percy Jackson and the Olympians',
 'The Sea of Monsters',
 "Demigods and Monsters: Your Favorite Authors on Rick Riordan's Percy Jackson and the Olympians Series",
 'Percy Jackson and the Olympians Boxed Set',
 'The Battle of the Labyrinth',
 'The Mark of Athena',
 "Percy Jackson's Greek Gods",
 'The Last Olympian',
 'The Demigod Files',
 'Switched',
 'The House of Hades',
 'The Demigod Diaries',
 'Percy Jackson & the Olympians:  The Ultimate Guide',
 "Percy Jackson's Greek Heroes",
 'The Blood of Olympus',
 'The Son of Neptune',
 'The Staff of Serapis',
 'The Hidden Oracle',
 'The Burning Maze',
 'Providence']